In [2]:
!pip install --quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.8 MB/s eta 0:00:00


In [45]:
import pandas as pd
import numpy as np
import openai
from openai.embeddings_utils import cosine_similarity
from sklearn.metrics import accuracy_score

EMBEDDING_MODEL = "text-embedding-ada-002"
OPENAI_API_KEY = "YOUR_API_KEY"

openai.api_key = OPENAI_API_KEY

reference_matrix_df = pd.read_csv("ENEM_Reference_Matrix.csv")
questions_df = pd.read_csv("ENEM_tagged_questions.csv")

In [46]:
reference_matrix_df.head()

,Discipline,Skill_Number,Skill_Description
0,"Linguagens, Códigos e suas Tecnologias",1,Identificar as diferentes linguagens e seus re...
1,"Linguagens, Códigos e suas Tecnologias",2,Recorrer aos conhecimentos sobre as linguagens...
2,"Linguagens, Códigos e suas Tecnologias",3,Relacionar informações geradas nos sistemas de...
3,"Linguagens, Códigos e suas Tecnologias",4,Reconhecer posições críticas aos usos sociais ...
4,"Linguagens, Códigos e suas Tecnologias",5,Associar vocábulos e expressões de um texto em...


In [47]:
questions_df.head()

,Year,Discipline,SG_AREA,TX_COR,CO_PROVA,Question_Number,Question_Text,Skill_Number
0,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 136 - Matemática e suas Tecnologias,Uma máquina em operação tem sua temperat...,25
1,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 137 - Matemática e suas Tecnologias,A World Series é a decisão do campeonat...,28
2,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 138 - Matemática e suas Tecnologias,O gráfico apresenta os totais de receitas e de...,27
3,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 139 - Matemática e suas Tecnologias,Um casal está reformando a cozinha de c...,11
4,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 140 - Matemática e suas Tecnologias,Foram convidadas 32 equipes para um torn...,3


## Zero-Shot Classification

In [40]:
def get_embedding(texts, model=EMBEDDING_MODEL):
   texts = [text.replace("\n", " ") for text in texts]

   embeddings = openai.Embedding.create(input = texts, model=model)['data']
   embeddings = [embedding['embedding'] for embedding in embeddings]

   return embeddings

In [49]:
reference_matrix_df['Skill_Embedding'] = get_embedding((reference_matrix_df['Discipline'] + " - " + reference_matrix_df['Skill_Description']).tolist())

def predict_skill(question_embedding, reference_matrix):
    # calculate cosine similarities between the question and all skills
    similarities = [cosine_similarity(question_embedding, skill_embedding) for skill_embedding in reference_matrix['Skill_Embedding']]

    # find the skill with the highest similarity
    top_skill_index = np.argmax(similarities)

    # return the predicted skill number
    return reference_matrix.iloc[top_skill_index]['Skill_Number']

# get embeddings for all questions
questions_df['Question_Embedding'] = get_embedding((questions_df['Discipline'] + " - " + questions_df['Question_Text']).tolist())

# predict skill for each question
questions_df['Predicted_Skill'] = questions_df['Question_Embedding'].apply(predict_skill, reference_matrix=reference_matrix_df)

# calculate accuracy
accuracy = accuracy_score(questions_df['Skill_Number'], questions_df['Predicted_Skill'])
print(f'Top-1 Accuracy: {accuracy}')

Top-1 Accuracy: 0.2222222222222222


In [50]:
questions_df

,Year,Discipline,SG_AREA,TX_COR,CO_PROVA,Question_Number,Question_Text,Skill_Number,Question_Embedding,Predicted_Skill
0,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 136 - Matemática e suas Tecnologias,Uma máquina em operação tem sua temperat...,25,"[-0.019639581441879272, -0.0029692708048969507...",13
1,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 137 - Matemática e suas Tecnologias,A World Series é a decisão do campeonat...,28,"[0.023165801540017128, -0.009591477923095226, ...",28
2,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 138 - Matemática e suas Tecnologias,O gráfico apresenta os totais de receitas e de...,27,"[0.00025632756296545267, 0.0007947381236590445...",26
3,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 139 - Matemática e suas Tecnologias,Um casal está reformando a cozinha de c...,11,"[0.03241261839866638, 0.014649772085249424, 0....",12
4,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 140 - Matemática e suas Tecnologias,Foram convidadas 32 equipes para um torn...,3,"[0.0075425500981509686, 0.007627444341778755, ...",5
5,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 141 - Matemática e suas Tecnologias,"Em um jogo de bingo, as cartelas contêm...",28,"[-0.013912173919379711, 0.012819628231227398, ...",28
6,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 142 - Matemática e suas Tecnologias,Uma montadora de automóveis divulgou que ...,5,"[0.005701425019651651, 0.026513945311307907, 0...",5
7,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 143 - Matemática e suas Tecnologias,Dentre as diversas planificações possíveis...,7,"[0.02105008065700531, 0.010307690128684044, 0....",7
8,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 144 - Matemática e suas Tecnologias,Cada número que identifica uma agência bancári...,1,"[0.0007412074482999742, 0.007528374437242746, ...",1
9,2022,Matemática e suas Tecnologias,MT,AZUL,1075,Questão 145 - Matemática e suas Tecnologias,O pacote básico de um jogo para smartph...,18,"[0.0072074392810463905, 0.01765856146812439, 0...",14
